In [11]:
import os
import time
from dotenv import load_dotenv
from pinecone import Pinecone
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain


In [12]:
load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key)
client = OpenAI()

In [13]:
index_name = "ia-sensor"

if not pc.has_index(index_name):
   pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
    
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

# Sem prompt

In [14]:
# Upload do PDFs
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Extrair texto dos documentos
texts = []
for doc in docs:
    texts.append(doc.page_content)

# Dividir texto em chunks
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

split_texts = text_splitter.split_documents(docs)

# Gera embeddings
embeddings = OpenAIEmbeddings()

# Criação do armazenamento vetorial no Pinecone
vectorstore = PineconeVectorStore.from_documents(
    documents=split_texts, 
    embedding=embeddings, 
    index_name=index_name 
)

# Configuração do modelo
llm = ChatOpenAI(   
    model='gpt-3.5-turbo',  
    temperature=0.0  
)  

#  Configuração de mecanismo de perguntas e respostas
retriever = vectorstore.as_retriever()

qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)  

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)

contexto = "\n".join(texts) 

# Consulta para buscar informações
pergunta = "Como faço para congifurar a central para o cliente?"

pergunta_com_contexto = f"Contexto:\n{contexto}\n\nPergunta: {pergunta}"

resposta = llm(pergunta_com_contexto)

display(Markdown(resposta.content)) 



C:\Users\Sensorweb2\AppData\Local\Temp\ipykernel_18700\979736543.py:60: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resposta = llm(pergunta_com_contexto)


Para configurar a central para o cliente, você pode seguir os seguintes passos:

1. Acesse o PUTTY e execute o comando sensorgw appConfig.
2. Confirme a configuração padrão do servidor.
3. Insira o nome do cliente no campo Client WebApp.
4. Preencha os campos Client Login e Client Password.
5. Verifique a conexão com o CheckConnection e CheckModem.
6. Defina a operadora no campo Carrier.
7. Salve as configurações com o comando sync.
8. Pare o serviço com sensorgw stop.

Além disso, você pode configurar o HAMACHI seguindo os passos:
1. Pare o serviço do HAMACHI com /etc/init.d/logmein-hamachi stop.
2. Remova as configurações prévias do HAMACHI com rm -Rf /var/lib/logmein-hamachi/.
3. Inicie o serviço do HAMACHI com /etc/init.d/logmein-hamachi start.
4. Faça o login no HAMACHI com hamachi login.
5. Defina o nome da central para o HAMACHI com hamachi set-nick.
6. Vincule a conta do HAMACHI com hamachi attach.
7. Conecte na conta do HAMACHI com hamachi do-join.

Esses são alguns passos básicos para configurar a central para o cliente. Lembre-se de sempre verificar as especificidades do cliente e adaptar as configurações conforme necessário.

# Com prompt

In [ ]:
# Upload do PDFs
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Extrair texto dos documentos
texts = []
for doc in docs:
    texts.append(doc.page_content)

# Dividir texto em chunks
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

split_texts = text_splitter.split_documents(docs)

# Gera embeddings
embeddings = OpenAIEmbeddings()

# Criação do armazenamento vetorial no Pinecone
vectorstore = PineconeVectorStore.from_documents(
    documents=split_texts, 
    embedding=embeddings, 
    index_name=index_name 
)

# Configuração do modelo
llm = ChatOpenAI(   
    model='gpt-3.5-turbo',  
    temperature=0.0  
)  

# Configuração de mecanismo de perguntas e respostas
retriever = vectorstore.as_retriever()

qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)  

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)

contexto = "\n".join(texts) 

# Prompt aprimorado para respostas detalhadas e passo a passo
prompt = f"""
Use apenas as informações abaixo para responder a pergunta do usuário. Seja detalhado, em formato de passo a passo.\n\n{context}

Baseado no seguinte contexto:
{contexto}

Responda à pergunta:
{pergunta}

Formato esperado:
1. Passo 1: [descrição detalhada]
2. Passo 2: [descrição detalhada]
...

Se a resposta não estiver no contexto acima, responda: 'Não encontrei essa informação no material disponível.'
"""

# Consulta para buscar informações
pergunta = "Como faço para configurar ao central para o cliente?"

pergunta_com_contexto = prompt

resposta = llm(pergunta_com_contexto)

display(Markdown(resposta.content))


Para configurar a central para o cliente, siga os seguintes passos:

1. Acesse o PUTTY com o IP da central em teste e faça o login com as credenciais:
   - Login as: root
   - Password: mcaUser

2. Verifique a versão da Central utilizando o comando:
   ```
   $ sensorgw version
   ```

3. Caso seja necessário atualizar a versão da Central, utilize o comando de atualização correspondente à versão atual da Central, conforme a lista de comandos de atualização fornecida no contexto.

4. Após atualizar a versão, reinicie a central com o comando:
   ```
   $ reboot
   ```

5. Teste o BUZZER com o comando:
   ```
   $ beep -l 300
   ```

6. Teste o RTC (Relógio em Tempo Real) com os comandos:
   - Para verificar a hora atual: 
     ```
     $ hwclock -r
     ```
   - Para atualizar o RTC: 
     ```
     $ hwclock -w
     ```

7. Verifique se o GATEWAY "DEBUG" está funcionando corretamente, observando a leitura de transmissores, repetidores e informações do GATEWAY pela central, utilizando o comando:
   ```
   $ tail -100f /home/sensorgw/application/conf/data/log_dac_win_service.log
   ```

8. Para configurar a conexão do modem, siga os passos específicos para a versão do hardware da Central:
   - Para Central Lizzy (hardware 2.0):
     ```
     $ sensorgw modem easy_on
     $ pon operadora
     $ tail -f /var/log/messages
     $ ping -I ppp0 clientes.sensorweb.com.br
     $ poff -a
     ```

9. Teste a conexão do modem com os comandos:
   ```
   $ sensorgw modemCheck
   $ sensorgw switchOnModem
   ```

10. Verifique o GPIO para garantir que o pino 7 esteja ligado, utilizando os comandos:
    ```
    $ gpio readall
    $ sensorgw switchOffModem
    $ gpio readall
    $ sensorgw switchOnModem
    ```

11. Realize o teste do DIAGNÓSTICO com um Pendrive na entrada USB de cima e aguarde o BEEP de confirmação ao atingir 100% no display.

12. Configure a central para as configurações padrões com os comandos:
    ```
    $ sensorgw appConfig
    $ sensorgw stop
    $ pidof java
    ```

13. Defina as informações de rede da central para IP Dinâmico (D.H.C.P) e acesse o navegador com o IP da central, utilizando as credenciais fornecidas.

14. Teste se a bateria mantém o hardware em funcionamento sem a fonte, desconectando a fonte da central.

15. Teste o botão ON/OFF da Central para garantir seu funcionamento adequado.

16. Feche o gabinete da central, limpe a superfície e insira os adesivos conforme necessário.

Lembre-se de seguir todos os passos com atenção e garantir que cada etapa seja concluída corretamente antes de prosseguir para a próxima.